In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from researchlib.single_import import *

Using GPU 0: GeForce RTX 2080 Ti
Driver: 418.56


In [3]:
train_loader = VisionDataset(vision.MNIST, 32, normalize=False)

In [4]:
g = builder([
    nn.Linear(110, 8*4*4),
    nn.BatchNorm1d(8*4*4),
    nn.LeakyReLU(),
    layer.Reshape((-1, 8, 4, 4)),
    AutoConvTransposeNet2d(8, 3, 'vgg', start_filter=128, min_filter=128, preact=False),
    nn.Conv2d(128, 1, 5),
    nn.Sigmoid(),
])  

8 128
128 128
128 128


In [5]:
d = builder([
#     layer.ConditionProjection(
#         builder([
#             nn.Linear(10, 1024),
#             nn.BatchNorm1d(1024),
#             nn.LeakyReLU(),
#             layer.Reshape((-1, 1, 32, 32)),
#         ]),
#         builder([
#             nn.ConstantPad2d(2, 0),
#         ])
#     ),
    nn.ConstantPad2d(2, 0),
    AutoConvNet2d(1, 5, 'vgg', start_filter=32, max_filter=128, flatten=True, preact=False),
    Auxiliary(builder([
        nn.Linear(128, 10),
        nn.LogSoftmax(-1)
    ])),
    nn.Linear(128, 1),
    nn.Sigmoid()
])

1 32
32 64
64 128
128 128
128 128


In [6]:
runner = Runner(GANModel(g, d, latent_vector_len=100, condition_vector_len=[10, None], condition_onehot=True),
                train_loader, 
                None, 
                'rmsprop', 
                GANLoss('vanilla', aux_loss=F.nll_loss),
                monitor_state='g_loss',
                monitor_mode='min')

In [7]:
runner.init_model('orthogonal')

Initialize to orthogonal : Linear(in_features=110, out_features=128, bias=True)
Initialize to orthogonal : Conv2d(8, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Initialize to orthogonal : Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Initialize to orthogonal : ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
Initialize to orthogonal : _UpSampling(
  (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (activator): ELU(alpha=1.0)
  (m): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
)
Initialize to orthogonal : _UpSampling(
  (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (activator): ELU(alpha=1.0)
  (m): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
)
Initialize to orthogonal : Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Initialize to orthogonal : ModuleList(
  (0): _UpSampling(
    (conv): Conv2d(128, 128, kernel_size=(3, 3), strid

Initialize to orthogonal : _ConvTransposeBlock2d(
  (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activator): ELU(alpha=1.0)
  (pooling_f): builder(
    (nnlist): ModuleList(
      (0): _UpSampling(
        (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (activator): ELU(alpha=1.0)
        (m): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
      )
      (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
  )
)
Initialize to orthogonal : Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Initialize to orthogonal : Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Initialize to orthogonal : ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
Initialize to orthogonal : _UpSampling(
  (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), paddin

Initialize to orthogonal : ModuleList(
  (0): _ConvTransposeBlock2d(
    (conv): Conv2d(8, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activator): ELU(alpha=1.0)
    (pooling_f): builder(
      (nnlist): ModuleList(
        (0): _UpSampling(
          (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (activator): ELU(alpha=1.0)
          (m): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
        )
        (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
  )
  (1): _ConvTransposeBlock2d(
    (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (activator): ELU(alpha=1.0)
    (pooling_f): builder(
      (nnlist): ModuleList(
        (0): _UpSampling(
          (conv): Conv2d(128

Initialize to orthogonal : Conv2d(384, 128, kernel_size=(1, 1), stride=(1, 1))
Initialize to orthogonal : _DownSampling(
  (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (a): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (c): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (red): Conv2d(384, 128, kernel_size=(1, 1), stride=(1, 1))
  (activator): ELU(alpha=1.0)
)
Initialize to orthogonal : _DownSampling(
  (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (a): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (c): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (red): Conv2d(384, 128, kernel_size=(1, 1), stride=(1, 1))
  (activator): ELU(alpha=1.0)
)
Initialize to orthogonal : _ConvBlock2d(
  (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activator): ELU(alpha

Initialize to orthogonal : builder(
  (nnlist): ModuleList(
    (0): _ConvBlock2d(
      (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activator): ELU(alpha=1.0)
      (pooling_f): _DownSampling(
        (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (a): AvgPool2d(kernel_size=2, stride=2, padding=0)
        (c): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (red): Conv2d(96, 32, kernel_size=(1, 1), stride=(1, 1))
        (activator): ELU(alpha=1.0)
      )
    )
    (1): _ConvBlock2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (activator): ELU(alpha=1.0)
      (pooling_f): _DownSampling(
        (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, cei

Initialize to orthogonal : builder(
  (nnlist): ModuleList(
    (0): ConstantPad2d(padding=(2, 2, 2, 2), value=0)
    (1): builder(
      (nnlist): ModuleList(
        (0): _ConvBlock2d(
          (conv): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (activator): ELU(alpha=1.0)
          (pooling_f): _DownSampling(
            (m): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
            (a): AvgPool2d(kernel_size=2, stride=2, padding=0)
            (c): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
            (red): Conv2d(96, 32, kernel_size=(1, 1), stride=(1, 1))
            (activator): ELU(alpha=1.0)
          )
        )
        (1): _ConvBlock2d(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_ru

In [8]:
runner.fit(4, 1e-4)

  Epochs   train_d_loss  train_g_loss 
    1*        0.0461        6.1223    


KeyboardInterrupt: 

In [ ]:
img = runner.model.sample(20, condition_data=list(range(10))*2)
import matplotlib.pyplot as plt
fig, arr = plt.subplots(2, 10, figsize=(20, 20))
for i in range(20):
    r = img[i][0].detach().cpu().numpy()#.transpose(1,2,0)
#     r -= r.min()
#     r /= r.max()
    arr[i//10][i%10].imshow(r, cmap='gray')
plt.show()

In [ ]:
runner.report()